## 爬取数据：
### 从UWAY上获取最新的对应发布版本query_list issue.csv

In [20]:
# import requests
import re
import datetime
import pandas as pd
import glob
import os
import sys
import matplotlib.pyplot as plt
import requests
import numpy as np

# 头部信息
headers = {
 'Host':"uway.youedata.com",
 'Accept-Language':"zh-CN,zh;q=0.8",
 'Accept-Encoding':"gzip, deflate",
 'Content-Type':"application/x-www-form-urlencoded",
 'Connection':"keep-alive",
 'Referer':"http://uway.youedata.com/uway/login",
 'User-Agent':"Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/46.0.2490.86 Safari/537.36"
}

print("[INFO] started...")
#login url
url = 'http://uway.youedata.com/uway/login'
#启用session，保存会话上下文
r_session = requests.Session()
#初次访问，获取网页内容
page = r_session.get(url)
#print(page.text)

#根据正则，获取auth——token，用于登录时，传入
reg = r'<meta name="csrf-token" content="(.+)" />'
csrf = re.findall(reg,page.text)[0]
print("[INFO] 成功获取到token："+csrf)

#form data，可以从chrome控制台得到，登陆时需要的form data
data = {
 "authenticity_token" : csrf,
 "username": "lilin",
 "password": "pas@123a"
 }
print("[INFO] 开始登陆...")
#登陆post action
response = r_session.post(url, data=data, headers=headers)
print("[INFO] 登陆OK")

#用同样的session，去获取csv 数据
res = r_session.get('http://uway.youedata.com/uway/projects/dataos-superba/issues.csv?query_id=99')
res.encoding = 'gb2312'

result_csv_filename = 'issues_'+datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")+".csv"

latest_issue = open(result_csv_filename,'a+')
latest_issue.write(res.text)
latest_issue.close()
print("[INFO] CSV 获取成功! file="+result_csv_filename)

print("[INFO] 开始处理csv文件...")

[INFO] started...
[INFO] 成功获取到token：gKN5nuVqGJm2er3mh5pD1gJCHyG1o4seGZp+HSC3HE3yOhHKZqPar6HKw0vwGkrSpKnvjA/rL2TRrzixy2mIDw==
[INFO] 开始登陆...
[INFO] 登陆OK
[INFO] CSV 获取成功! file=issues_2020-12-08_15-25-40-642662.csv
[INFO] 开始处理csv文件...


## 读取issue.csv

In [21]:
#读取csv文件
data_frame = pd.read_csv(result_csv_filename)

## 数据清洗-缺失值处理

In [22]:
#清洗空值，特别是目标版本,注意，数据清洗不会在当前dataframe，需要保存到新的dataframe
data_frame = data_frame.fillna(value='未指定')

## 数据筛选-不需要的数据处理，比如某些目标版本的缺陷不需要

In [23]:
#python的关键字and和or对布尔值数组并没有用，需要用|和&来代替
#为了更加灵活，增添一个不需要的目标版本list
tbd_target_version_list = ['雨燕mPaas 1.4.3','雨燕mPaas 1.4.2','雨燕mPaas 1.4.1']
for i in tbd_target_version_list:
    data_frame = data_frame.loc[(data_frame['目标版本'] != i)]
tbd_bug_list = ['【易道169升级】升级169后，dataos后端内存溢出概率大','【文档】文档缺陷']
for i in tbd_bug_list:
   data_frame = data_frame.loc[(data_frame['主题'] != i)]

In [24]:
#列出特定关注的卡
res=data_frame[data_frame['主题'].str.contains("龙湖")]
#spike_cols =[x for x in data.columns[data.columns.str.contains('bug')]]
#spike_cols
res

,#,跟踪,状态,优先级,主题,项目,处理人,指派给,目标版本,发布版本,作者,创建于
1,31224,缺陷,新建,提示bug,【龙湖】【服务中心】服务页面，服务配置处，主机host，建议修改为主机域名,服务中心（黄欣）,未指定,翟小童,服务中心第二版v1.8.1,V2.0.5,马文龙,2020-12-03 15:03
2,31218,缺陷,新建,提示bug,【龙湖】【服务中心】编辑服务，缺少提示【主机域名 +端口号】连通性,服务中心（黄欣）,未指定,翟小童,服务中心第二版v1.8.1,V2.0.5,马文龙,2020-12-03 14:48
3,31215,缺陷,新建,提示bug,【龙湖】【服务中心】新建API页面、api删除，部分api的小写需修改为大写,服务中心（黄欣）,未指定,翟小童,服务中心第二版v1.8.1,V2.0.5,马文龙,2020-12-03 14:33
4,31211,缺陷,新建,提示bug,【龙湖】【服务中心】服务列表，查询结果为空，页面提示，提示不准确,服务中心（黄欣）,未指定,翟小童,服务中心第二版v1.8.1,V2.0.5,马文龙,2020-12-03 14:24
5,31209,缺陷,新建,提示bug,【龙湖】【服务中心】服务详情中，关联API处，查询条件提示，API名需改为：API标题,服务中心（黄欣）,未指定,翟小童,服务中心第二版v1.8.1,V2.0.5,马文龙,2020-12-03 14:11
6,31208,缺陷,新建,提示bug,【龙湖】【服务中心】服务详情中，“说明链接”字段显示为“说明文档”,服务中心（黄欣）,未指定,翟小童,服务中心第二版v1.8.1,V2.0.5,马文龙,2020-12-03 14:08
7,31205,缺陷,新建,提示bug,【龙湖】【服务中心】重复上传同一大于10M服务说明doc文档时，页面无提示,服务中心（黄欣）,未指定,翟小童,服务中心第二版v1.8.1,V2.0.5,马文龙,2020-12-03 14:02


## 数据透视表聚合和分组数据

In [25]:
data = pd.pivot_table(data_frame,index=['目标版本'],columns=['优先级'],values=['#'],aggfunc=np.count_nonzero,fill_value=0)

In [26]:
data

#            
优先级           一般bug 严重bug 提示bug
目标版本                           
2.0.0             1     0     0
中台桌面V1.6          1     0     0
安全中心V1.4.3        0     1     0
服务中心第二版v1.8.1     0     0     7
未指定               0     0     1

## 数据表重构：重构成需要的表

## 拆分tuple元组columns，简化dataframe

In [27]:
data = data.pop("#")

##  构造要求的Columns 

In [28]:
column_list = ['致命bug','严重bug','一般bug','提示bug']
data = data.reindex(columns = column_list )
#把缺失值替换为0.
data = data.replace(np.nan, 0)

## 数据类型转换到整型，默认是float

In [29]:
data = data.astype(int)

## 行列求和

In [30]:
data.loc['整个DataOS'] = data.apply(lambda x: x.sum())
data['全部缺陷'] = data.apply(lambda x: x.sum(), axis=1)

## 调整index 整个DataOS到首行

In [31]:
#获取默认index列表
index = data.index.tolist()

#把“整个DataOS”加到列表首位
index.insert(0,'整个DataOS')

#把原来最末行的“整个DataOS”删除
index.pop()

#重建行索引index
data = data.reindex(index )

## 通过lambda函数列运算计算DI值

In [32]:
#data_frame['DI值'] = data_frame.apply(lambda x: x['致命bug']*10+x['严重bug']*3+x['一般bug']*1+x['提示bug']*0.1,axis=1)
#更简单优雅的lambda列运算，columns顺序调整好，直接用columns的数字来代替文字。
data['DI值'] = data.apply(lambda x: x[0]*10+x[1]*3+x[2]*1+x[3]*0.1,axis=1)

data

优先级,致命bug,严重bug,一般bug,提示bug,全部缺陷,DI值
目标版本,,,,,,
整个DataOS,0,1,2,8,11,5.8
2.0.0,0,0,1,0,1,1.0
中台桌面V1.6,0,0,1,0,1,1.0
安全中心V1.4.3,0,1,0,0,1,3.0
服务中心第二版v1.8.1,0,0,0,7,7,0.7
未指定,0,0,0,1,1,0.1


## 列出DI值关注的bug，默认是致命bug和严重bug

In [33]:
#data = data_frame.loc[((data_frame['优先级'] == '致命bug') | (data_frame['优先级'] == '严重bug')|(data_frame['优先级'] == '一般bug')), :]
data = data_frame.loc[((data_frame['优先级'] == '致命bug') | (data_frame['优先级'] == '严重bug')), :]

data = data.groupby(['优先级','目标版本','状态','#','指派给','处理人','主题'])['#'].agg(严重致命bug=np.count_nonzero)
data

,,,,,,,严重致命bug
优先级,目标版本,状态,#,指派给,处理人,主题,
严重bug,安全中心V1.4.3,新建,31461,韦君,未指定,【安全中心-功能帮助】点击功能帮助，查看功能帮助详情失败，提示：Knife4j文档请求异常,1


In [34]:

data = data_frame.loc[((data_frame['目标版本'] == '中台桌面V1.2') | (data_frame['目标版本'] == '互联网桌面 V1.0')|(data_frame['目标版本'] == '桌面管理V1.4')), :]

data = data.groupby(['目标版本','优先级','状态','#','指派给','处理人','主题'])['#'].agg(严重致命bug=np.count_nonzero)
data

,严重致命bug


In [35]:
data = data_frame.loc[((data_frame['目标版本'] == '数据中心-v1.3.2') ), :]

data = data.groupby(['目标版本','优先级','状态','#','指派给','处理人','主题'])['#'].agg(严重致命bug=np.count_nonzero)
data

,严重致命bug


In [36]:
data = data_frame.loc[((data_frame['状态'] == '已解决')), :]

data = data.groupby(['目标版本','优先级','状态','#','指派给','处理人','主题'])['#'].agg(严重致命bug=np.count_nonzero)
data

,严重致命bug


In [37]:
data = data_frame.loc[((data_frame['优先级'] == '一般bug')), :]

data = data.groupby(['目标版本','优先级','状态','#','指派给','处理人','作者','主题'])['#'].agg(严重致命bug=np.count_nonzero)
data

,,,,,,,,严重致命bug
目标版本,优先级,状态,#,指派给,处理人,作者,主题,
2.0.0,一般bug,新建,29069,杨柳,"杨兵, 滕聪",张小华,【云市场】组件下载注册失败,1
中台桌面V1.6,一般bug,新建,31429,许济川,未指定,马文龙,【中台桌面】移动端绑定自由体系应用，显示为登录页面,1


In [38]:
data = data_frame.loc[((data_frame['优先级'] == '提示bug')), :]

data = data.groupby(['目标版本','优先级','状态','#','指派给','处理人','作者','主题'])['#'].agg(严重致命bug=np.count_nonzero)
data

严重致命bug
目标版本          优先级   状态   #     指派给 处理人 作者  主题                                                         
服务中心第二版v1.8.1 提示bug 新建   31205 翟小童 未指定 马文龙 【龙湖】【服务中心】重复上传同一大于10M服务说明doc文档时，页面无提示                     1
                         31208 翟小童 未指定 马文龙 【龙湖】【服务中心】服务详情中，“说明链接”字段显示为“说明文档”                         1
                         31209 翟小童 未指定 马文龙 【龙湖】【服务中心】服务详情中，关联API处，查询条件提示，API名需改为：API标题               1
                         31211 翟小童 未指定 马文龙 【龙湖】【服务中心】服务列表，查询结果为空，页面提示，提示不准确                          1
                         31215 翟小童 未指定 马文龙 【龙湖】【服务中心】新建API页面、api删除，部分api的小写需修改为大写                    1
                         31218 翟小童 未指定 马文龙 【龙湖】【服务中心】编辑服务，缺少提示【主机域名 +端口号】连通性                         1
                         31224 翟小童 未指定 马文龙 【龙湖】【服务中心】服务页面，服务配置处，主机host，建议修改为主机域名                     1
未指定           提示bug 延期处理 15012 刘林  罗俊  刘林  【数据中心】FTP连通后，同步数据页面展开文件夹后只勾选文件夹下的单一文件提交，提交后同步的文...        1